# eda002
各列が一致したときにPoIが一致する確率

In [1]:
import os
import sys
sys.path.append('/home/kaggler/.local/lib/python3.8/site-packages')
from dotenv import load_dotenv
load_dotenv
sys.path.append(os.getenv('UTILS_PATH'))
import pandas as pd
import matplotlib.pyplot as plt; plt.style.use("ggplot")
import seaborn as sns
from sklearn.metrics.pairwise import haversine_distances
import itertools

In [2]:
import builtins
import types

def imports():
    for name, val in globals().items():
        # module imports
        if isinstance(val, types.ModuleType):
            yield name, val

            # functions / callables
        if hasattr(val, '__call__'):
            yield name, val


def noglobal(f):
    '''
    ref: https://gist.github.com/raven38/4e4c3c7a179283c441f575d6e375510c
    '''
    return types.FunctionType(f.__code__,
                              dict(imports()),
                              f.__name__,
                              f.__defaults__,
                              f.__closure__
                              )

In [3]:
INPUT_DIR = os.getenv('INPUT_DIR')
OUTPUT_DIR = os.getenv('OUTPUT_DIR')
#exp_name = os.path.dirname(__file__).split('/')[-1]
exp_name = 'eda002'
os.makedirs(OUTPUT_DIR + exp_name, exist_ok=True)

In [4]:
train = pd.read_csv(INPUT_DIR + 'train.csv')

In [5]:
train['lat_lon'] = train['latitude'].astype(str) + ' _ ' + train['longitude'].astype(str)

In [6]:
train.columns

Index(['id', 'name', 'latitude', 'longitude', 'address', 'city', 'state',
       'zip', 'country', 'url', 'phone', 'categories', 'point_of_interest',
       'lat_lon'],
      dtype='object')

In [7]:
def make_match_data(col):
    id1s = []
    id2s = []
    for poi, df in train[["id", col]].groupby(col):
        if len(df) > 1:
            for id1,id2 in itertools.combinations(df['id'].values, 2):
                id1s.append(id1)
                id2s.append(id2)

    match_list = pd.DataFrame()
    match_list['id1'] = id1s
    match_list['id2'] = id2s

    train1 = train[['id', col, 'point_of_interest']].copy()
    train1.columns = train1.columns + '1'
    train2 = train[['id', col, 'point_of_interest']].copy()
    train2.columns = train2.columns + '2'

    match_list = match_list.merge(train1, on='id1', how='inner')
    match_list = match_list.merge(train2, on='id2', how='inner')
    match_list['poi_match'] = (match_list['point_of_interest1'] == match_list['point_of_interest2']).astype(int)
    match_rate = match_list['poi_match'].mean()
    print(col, 'record = ', len(match_list), '  match_rate = ' '{:.5f}'.format(match_rate*100) + '%')

    return match_list

In [8]:
print('各列が一致したときにPoIが一致する確率')
name_match = make_match_data('name')
latlon_match = make_match_data('lat_lon')
address_match = make_match_data('address')
# city_match = make_match_data('city')
# state_match = make_match_data('state')
zip_match = make_match_data('zip')
url_match = make_match_data('url')
phone_match = make_match_data('phone')

各列が一致したときにPoIが一致する確率
name record =  45930383   match_rate = 0.36925%
lat_lon record =  1046679   match_rate = 0.89196%
address record =  991892   match_rate = 6.73833%
zip record =  22589314   match_rate = 0.50047%
url record =  4186435   match_rate = 0.54660%
phone record =  2972318   match_rate = 0.99004%


In [10]:
latlon_match[latlon_match['poi_match']==0]

,id1,id2,lat_lon1,point_of_interest1,lat_lon2,point_of_interest2,poi_match
0,E_679c4397d77d34,E_b9d90cb233c3a9,-0.0245 _ 109.337,P_a4b427e8f2e263,-0.0245 _ 109.337,P_8b6027f19d110c,0
1,E_55a15d50697e9d,E_ed6c1d07465b43,-0.051389 _ 109.345,P_a6b8829d7f4a11,-0.051389 _ 109.345,P_1b480ccdabec74,0
2,E_55a15d50697e9d,E_fca2866a4a7c02,-0.051389 _ 109.345,P_a6b8829d7f4a11,-0.051389 _ 109.345,P_0570eb80d48430,0
3,E_ed6c1d07465b43,E_fca2866a4a7c02,-0.051389 _ 109.345,P_1b480ccdabec74,-0.051389 _ 109.345,P_0570eb80d48430,0
4,E_1d74b3a1c0f6b1,E_8e2d6ea5d8449f,-0.051713 _ 109.3452314,P_d017c73cc32b1e,-0.051713 _ 109.3452314,P_0a6d32f1eba6cd,0
...,...,...,...,...,...,...,...
1046663,E_602f882a73dc31,E_924f0c3506cdc6,7.880115032196045 _ 98.39729309082033,P_c8281d0893715a,7.880115032196045 _ 98.39729309082033,P_2f7c3a636641bc,0
1046668,E_0ad0bedb818a44,E_7918566a24a5e4,8.47821 _ 124.64776,P_9be1463df65ce0,8.47821 _ 124.64776,P_e5df8df23cee8f,0
1046671,E_21175d1a25fc95,E_895f5ec8cae27a,8.657967567443848 _ 99.37673950195312,P_ebb636dfa89010,8.657967567443848 _ 99.37673950195312,P_99145199a6e76e,0
1046674,E_7774f1dbe46501,E_a869462331ef39,9.3553011 _ 99.1697286,P_5ad31fec04521b,9.3553011 _ 99.1697286,P_75e8bd3c71c5e8,0


In [9]:
address_match[address_match['poi_match']==0]

,id1,id2,address1,point_of_interest1,address2,point_of_interest2,poi_match
2,E_d033094a53159b,E_e6bad213d6e742,"""Centro Universitario Regional Norte \""San Isi...",P_adb4a34eec463c,"""Centro Universitario Regional Norte \""San Isi...",P_fb314903b07b00,0
3,E_278f0c2c960188,E_326084e4520cc3,"""Colegiul Dobrogean \""Spiru C. Haret\""""",P_9680df52160aa5,"""Colegiul Dobrogean \""Spiru C. Haret\""""",P_ad4fef30930edc,0
4,E_278f0c2c960188,E_5cccfbb49e6d29,"""Colegiul Dobrogean \""Spiru C. Haret\""""",P_9680df52160aa5,"""Colegiul Dobrogean \""Spiru C. Haret\""""",P_a46dac28e58d61,0
5,E_326084e4520cc3,E_5cccfbb49e6d29,"""Colegiul Dobrogean \""Spiru C. Haret\""""",P_ad4fef30930edc,"""Colegiul Dobrogean \""Spiru C. Haret\""""",P_a46dac28e58d61,0
6,E_278f0c2c960188,E_9304e8e7105f17,"""Colegiul Dobrogean \""Spiru C. Haret\""""",P_9680df52160aa5,"""Colegiul Dobrogean \""Spiru C. Haret\""""",P_7de4c1df578ae4,0
...,...,...,...,...,...,...,...
991887,E_6430232f7bee28,E_edcb3263e59b0e,흑석동,P_ea0bc52f6f8ddb,흑석동,P_77237aa02329c6,0
991888,E_84efdac363caea,E_edcb3263e59b0e,흑석동,P_04645f7369a9d5,흑석동,P_77237aa02329c6,0
991889,E_241ea10c0a019b,E_565b8c43114332,🔥微信🔥n136094🔥微信🔥,P_060ec09ab47b71,🔥微信🔥n136094🔥微信🔥,P_5df5bf76014d05,0
991890,E_241ea10c0a019b,E_dabc2f6039be11,🔥微信🔥n136094🔥微信🔥,P_060ec09ab47b71,🔥微信🔥n136094🔥微信🔥,P_0b731da9b5cb4c,0
